In [1]:
# python3 /home/rnb/Projects/rnb-planning/src/pkg/planning/filtering/lattice_model/shared_lattice_predictor.py --rtype panda

# import SharedArray as sa
# ROBOT_TYPE_NAME = 'indy7gripper'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

In [1]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

DAT_ROOT = "stowing-deep"
RES_ROOT = "stowing-deep-result"

TEST_DATA_PATH = os.path.join(DATA_PATH, DAT_ROOT)
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, RES_ROOT)
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_count", "failed_MPs", "IK_count", "failed_IKs", "success"]

# obj1

In [2]:
RTYPE = "indy7gripper"
DAT_DIR = '20210907-173519'
FILE_OPTION = "obj_1"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['None', 'Tool', 'ToolReach', 'Full'] #,'Pairwise']
# CNAME_LIST = ['Full'] #,'Pairwise']
VISUALIZE = False
PLAY_RESULT = False
SHOW_STATE = False

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100

GRASP_SAMPLE = 30
STABLE_SAMPLE = 50
SEARCH_SAMPLE_RATIO= 100

### test

In [3]:
I_START = 0
I_END = 20
N_DAT = I_END-I_START

In [4]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 4.8     3.8     5.2     4.7     -------
========== 1 / 20 (eta: 18.4 / 368.8 s) =========
------- 17.3    3.9     4.4     4.6     -------
========== 2 / 20 (eta: 48.6 / 486.3 s) =========
------- 12.5    6.4     4.5     5.3     -------
========== 3 / 20 (eta: 77.4 / 515.9 s) =========
------- 4.0     3.8     6.5     6.3     -------
========== 4 / 20 (eta: 98.0 / 490.1 s) =========
------- 9.5     4.5     4.4     4.8     -------
========== 5 / 20 (eta: 121.2 / 484.6 s) =========
------- 19.6    5.5     4.8     4.8     -------
========== 6 / 20 (eta: 155.8 / 519.4 s) =========
------- 4.2     10.3    4.3     5.6     -------
========== 7 / 20 (eta: 180.2 / 514.8 s) =========
------- 6.5     3.9     4.6     4.4     -------
========== 8 / 20 (eta: 199.5 / 498.7 s) =========
------- 4.0     6.5     4.4     7.0     -------
========== 9 / 20 (eta: 221.3 / 491.8 s) =========
------- 10.8    8.3     4.7     4.6     -------
========== 10 / 20 (eta: 249.7 / 499.3 s) =========
------- 4.0  

# obj1 obs5

In [5]:
RTYPE = "indy7gripper"
# DAT_DIR = '20210907-071329'
FILE_OPTION = "obj_1_obs3"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# # CNAME_LIST = ['Full'] #,'Pairwise']
# VISUALIZE = False
# PLAY_RESULT = False
# SHOW_STATE = False

# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# MAX_ITER = 10000

# GRASP_SAMPLE = 30
# STABLE_SAMPLE = 50
# SEARCH_SAMPLE_RATIO= 100


### test

In [6]:
I_START = 0
I_END = 20
N_DAT = I_END-I_START

In [7]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 103.4   116.3   5.0     112.0   -------
========== 1 / 20 (eta: 336.7 / 6734.3 s) =========
------- 4.5     7.9     5.3     5.9     -------
========== 2 / 20 (eta: 360.2 / 3602.3 s) =========
------- 8.5     9.4     6.8     5.1     -------
========== 3 / 20 (eta: 390.0 / 2600.3 s) =========
------- 13.6    8.5     5.1     4.9     -------
========== 4 / 20 (eta: 422.2 / 2110.8 s) =========
------- 4.5     5.7     20.1    5.6     -------
========== 5 / 20 (eta: 458.2 / 1832.6 s) =========
------- 4.3     4.1     5.4     4.6     -------
========== 6 / 20 (eta: 476.6 / 1588.5 s) =========
------- 30.5    4.1     9.8     5.1     -------
========== 7 / 20 (eta: 526.1 / 1503.2 s) =========
------- 23.2    5.1     6.0     5.3     -------
========== 8 / 20 (eta: 565.7 / 1414.3 s) =========
------- 103.4   4.8     10.9    5.5     -------
========== 9 / 20 (eta: 690.4 / 1534.2 s) =========
------- 6.4     4.6     8.3     4.9     -------
========== 10 / 20 (eta: 714.6 / 1429.2 s) =========

# obj1 obs5

In [8]:
RTYPE = "indy7gripper"
# DAT_DIR = '20210907-071329'
FILE_OPTION = "obj_1_obs5"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# # CNAME_LIST = ['Full'] #,'Pairwise']
# VISUALIZE = False
# PLAY_RESULT = False
# SHOW_STATE = False

# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# MAX_ITER = 10000

# GRASP_SAMPLE = 30
# STABLE_SAMPLE = 50
# SEARCH_SAMPLE_RATIO= 100


### test

In [9]:
I_START = 0
I_END = 20
N_DAT = I_END-I_START

In [10]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 10.7    7.8     5.2     5.2     -------
========== 1 / 20 (eta: 28.9 / 578.5 s) =========
------- 8.8     4.9     4.9     4.5     -------
========== 2 / 20 (eta: 52.1 / 520.7 s) =========
------- 82.3    6.4     5.4     6.2     -------
========== 3 / 20 (eta: 152.4 / 1016.1 s) =========
------- 103.5   37.6    113.1   110.6   -------
========== 4 / 20 (eta: 517.3 / 2586.6 s) =========
------- 104.4   24.3    40.1    110.5   -------
========== 5 / 20 (eta: 796.6 / 3186.6 s) =========
------- 10.1    11.8    36.3    14.8    -------
========== 6 / 20 (eta: 869.7 / 2899.0 s) =========
------- 8.8     5.8     5.1     4.8     -------
========== 7 / 20 (eta: 894.2 / 2554.9 s) =========
------- 108.4   19.9    105.0   71.1    -------
========== 8 / 20 (eta: 1198.5 / 2996.3 s) =========
------- 10.2    5.4     6.7     5.2     -------
========== 9 / 20 (eta: 1226.1 / 2724.6 s) =========
------- 103.8   9.8     104.0   106.9   -------
========== 10 / 20 (eta: 1550.6 / 3101.1 s) =========


# obj3

In [11]:
RTYPE = "indy7gripper"
# DAT_DIR = '20210907-071329'
FILE_OPTION = "obj_3"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# GRASP_SAMPLE = 100
# VISUALIZE = True
# PLAY_RESULT = True
# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# SHOW_STATE = False


In [12]:
I_START=0
I_END = 20
N_DAT = I_END-I_START

### test

In [13]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 5.4     5.1     4.6     6.1     -------
========== 1 / 20 (eta: 21.2 / 424.1 s) =========
------- 26.3    3.9     5.5     5.3     -------
========== 2 / 20 (eta: 62.3 / 623.2 s) =========
------- 107.7   9.7     5.3     104.2   -------
========== 3 / 20 (eta: 289.2 / 1928.2 s) =========
------- 13.1    10.0    6.3     4.9     -------
========== 4 / 20 (eta: 323.6 / 1617.9 s) =========
------- 106.3   127.4   7.8     7.6     -------
========== 5 / 20 (eta: 572.7 / 2290.9 s) =========
------- 10.2    5.0     6.7     5.2     -------
========== 6 / 20 (eta: 599.8 / 1999.5 s) =========
------- 16.4    5.9     4.9     4.9     -------
========== 7 / 20 (eta: 631.9 / 1805.5 s) =========
------- 10.1    4.9     4.6     14.3    -------
========== 8 / 20 (eta: 665.8 / 1664.5 s) =========
------- 15.7    9.4     4.5     4.9     -------
========== 9 / 20 (eta: 700.4 / 1556.4 s) =========
------- 15.0    10.3    103.9   4.7     -------
========== 10 / 20 (eta: 834.2 / 1668.3 s) =========
---

# obj3_pole

In [20]:
RTYPE = "indy7gripper"
# DAT_DIR = '20210907-071329'
FILE_OPTION = "obj_3_pole"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3_pole"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# GRASP_SAMPLE = 100
# VISUALIZE = True
# PLAY_RESULT = True
# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# SHOW_STATE = False


In [21]:
I_START=0
I_END = 20
N_DAT = I_END-I_START

### test

In [22]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 103.7   103.7   76.7    95.1    -------
========== 1 / 20 (eta: 379.2 / 7584.3 s) =========


KeyboardInterrupt: 

# obj3_hard

In [17]:
RTYPE = "indy7gripper"
# DAT_DIR = '20210907-071329'
# DAT_DIR = '20210907-071329'
# DAT_DIR = '20210702-152658'
FILE_OPTION = "obj_3_hard"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3_pole"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['ToolReach','Full'] #,'Pairwise']
# # CNAME_LIST = ['Full'] #,'Pairwise']
# GRASP_SAMPLE = 100
# VISUALIZE = False
# PLAY_RESULT = False
# SHOW_STATE = False

# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# MAX_ITER = 100

In [18]:
I_START=0
I_END = 20
N_DAT = I_END-I_START

### test

In [19]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 103.5   106.3   9.3     103.9   -------
========== 1 / 20 (eta: 323.0 / 6460.4 s) =========
------- 13.3    5.9     7.0     5.9     -------
========== 2 / 20 (eta: 355.1 / 3551.5 s) =========
------- 11.7    5.6     7.8     75.7    -------
========== 3 / 20 (eta: 456.0 / 3039.9 s) =========
------- 11.5    4.8     103.9   7.8     -------
========== 4 / 20 (eta: 584.0 / 2919.9 s) =========
------- 15.0    57.1    105.6   142.8   -------
========== 5 / 20 (eta: 904.4 / 3617.5 s) =========
------- 12.8    104.2   6.2     8.1     -------
========== 6 / 20 (eta: 1035.8 / 3452.5 s) =========
------- 66.9    4.9     5.3     8.2     -------
========== 7 / 20 (eta: 1121.2 / 3203.4 s) =========
------- 103.6   103.5   104.0   103.9   -------
========== 8 / 20 (eta: 1536.2 / 3840.6 s) =========
------- 103.4   103.4   110.1   128.1   -------
========== 9 / 20 (eta: 1981.2 / 4402.7 s) =========
------- 5.5     5.1     8.4     10.4    -------
========== 10 / 20 (eta: 2010.5 / 4021.1 s) ====

# obj1 obs0

In [3]:
RTYPE = "indy7gripper"
# DAT_DIR = '20210907-071329'
FILE_OPTION = "obj_1c_obs0"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# # CNAME_LIST = ['Full'] #,'Pairwise']
# VISUALIZE = False
# PLAY_RESULT = False
# SHOW_STATE = False

# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# MAX_ITER = 10000

# GRASP_SAMPLE = 30
# STABLE_SAMPLE = 50
# SEARCH_SAMPLE_RATIO= 100


### test

In [4]:
I_START = 0
I_END = 40
N_DAT = I_END-I_START

In [5]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 7.1     5.0     4.9     5.3     -------
========== 1 / 40 (eta: 22.2 / 888.1 s) =========
------- 9.3     5.2     5.0     5.5     -------
========== 2 / 40 (eta: 47.2 / 943.6 s) =========
------- 6.0     5.1     7.2     6.7     -------
========== 3 / 40 (eta: 72.1 / 961.1 s) =========
------- 6.3     5.9     5.3     5.3     -------
========== 4 / 40 (eta: 94.9 / 948.6 s) =========
------- 5.1     12.7    4.9     5.5     -------
========== 5 / 40 (eta: 123.0 / 984.4 s) =========
------- 12.8    14.7    5.1     5.4     -------
========== 6 / 40 (eta: 161.0 / 1073.6 s) =========
------- 4.7     5.4     6.0     5.6     -------
========== 7 / 40 (eta: 182.7 / 1044.1 s) =========
------- 12.2    6.6     6.2     5.6     -------
========== 8 / 40 (eta: 213.3 / 1066.4 s) =========
------- 11.0    13.4    5.3     5.6     -------
========== 9 / 40 (eta: 248.6 / 1105.0 s) =========
------- 6.5     5.6     5.0     5.8     -------
========== 10 / 40 (eta: 271.6 / 1086.2 s) =========
------- 

# obj1 obs3

In [4]:
RTYPE = "indy7gripper"
# DAT_DIR = '20210907-071329'
FILE_OPTION = "obj_1c_obs3"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# # CNAME_LIST = ['Full'] #,'Pairwise']
# VISUALIZE = False
# PLAY_RESULT = False
# SHOW_STATE = False

# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# MAX_ITER = 10000

# GRASP_SAMPLE = 30
# STABLE_SAMPLE = 50
# SEARCH_SAMPLE_RATIO= 100


### test

In [5]:
I_START = 0
I_END = 40
N_DAT = I_END-I_START

In [6]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 6.5     4.7     5.9     5.8     -------
========== 1 / 40 (eta: 23.0 / 921.8 s) =========
------- 12.7    11.1    5.1     5.4     -------
========== 2 / 40 (eta: 57.3 / 1146.5 s) =========
------- 4.8     5.0     5.5     11.3    -------
========== 3 / 40 (eta: 83.9 / 1118.2 s) =========
------- 7.8     6.7     4.7     5.7     -------
========== 4 / 40 (eta: 108.8 / 1087.9 s) =========
------- 7.4     4.7     7.9     6.7     -------
========== 5 / 40 (eta: 135.4 / 1083.5 s) =========
------- 13.2    5.9     12.1    11.9    -------
========== 6 / 40 (eta: 178.5 / 1190.1 s) =========
------- 103.7   6.5     7.7     7.0     -------
========== 7 / 40 (eta: 303.4 / 1733.4 s) =========
------- 6.7     5.4     5.8     6.2     -------
========== 8 / 40 (eta: 327.4 / 1637.2 s) =========
------- 5.5     4.3     4.9     10.5    -------
========== 9 / 40 (eta: 352.6 / 1567.2 s) =========
------- 5.1     8.1     11.0    16.2    -------
========== 10 / 40 (eta: 393.0 / 1572.1 s) =========
---

# obj1 obs5

In [6]:
RTYPE = "indy7gripper"
# DAT_DIR = '20210907-071329'
FILE_OPTION = "obj_1c_obs5"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# # CNAME_LIST = ['Full'] #,'Pairwise']
# VISUALIZE = False
# PLAY_RESULT = False
# SHOW_STATE = False

# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# MAX_ITER = 10000

# GRASP_SAMPLE = 30
# STABLE_SAMPLE = 50
# SEARCH_SAMPLE_RATIO= 100


### test

In [7]:
I_START = 0
I_END = 40
N_DAT = I_END-I_START

In [8]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 5.3     8.0     6.6     5.7     -------
========== 1 / 40 (eta: 25.6 / 1025.0 s) =========
------- 16.4    19.4    20.9    7.3     -------
========== 2 / 40 (eta: 89.6 / 1792.7 s) =========
------- 106.6   28.5    23.9    14.2    -------
========== 3 / 40 (eta: 262.8 / 3504.1 s) =========
------- 104.2   107.1   49.7    16.2    -------
========== 4 / 40 (eta: 540.0 / 5399.5 s) =========
------- 13.9    6.0     6.4     12.5    -------
========== 5 / 40 (eta: 578.9 / 4630.8 s) =========
------- 25.8    14.9    7.6     5.6     -------
========== 6 / 40 (eta: 632.7 / 4218.3 s) =========
------- 111.4   104.0   114.8   108.4   -------
========== 7 / 40 (eta: 1071.5 / 6122.8 s) =========
------- 5.7     8.1     12.2    9.2     -------
========== 8 / 40 (eta: 1106.7 / 5533.4 s) =========
------- 106.1   11.0    110.3   112.4   -------
========== 9 / 40 (eta: 1446.5 / 6428.9 s) =========
------- 10.6    7.2     8.3     5.5     -------
========== 10 / 40 (eta: 1478.0 / 5912.1 s) =======

# obj1 obs7

In [9]:
RTYPE = "indy7gripper"
# DAT_DIR = '20210907-071329'
FILE_OPTION = "obj_1c_obs7"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# # CNAME_LIST = ['Full'] #,'Pairwise']
# VISUALIZE = False
# PLAY_RESULT = False
# SHOW_STATE = False

# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# MAX_ITER = 10000

# GRASP_SAMPLE = 30
# STABLE_SAMPLE = 50
# SEARCH_SAMPLE_RATIO= 100


### test

In [10]:
I_START = 0
I_END = 40
N_DAT = I_END-I_START

In [11]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 12.6    12.6    5.1     7.2     -------
========== 1 / 40 (eta: 37.6 / 1502.9 s) =========
------- 11.6    11.8    6.9     9.4     -------
========== 2 / 40 (eta: 77.3 / 1545.7 s) =========
------- 25.5    10.9    5.4     24.5    -------
========== 3 / 40 (eta: 143.5 / 1913.9 s) =========
------- 5.7     10.8    10.9    5.8     -------
========== 4 / 40 (eta: 176.7 / 1767.5 s) =========
------- 11.7    5.2     10.9    20.9    -------
========== 5 / 40 (eta: 225.5 / 1804.0 s) =========
------- 19.8    6.6     5.8     6.2     -------
========== 6 / 40 (eta: 263.8 / 1759.0 s) =========
------- 8.1     6.2     5.0     8.7     -------
========== 7 / 40 (eta: 291.9 / 1667.8 s) =========
------- 8.2     6.7     10.7    12.4    -------
========== 8 / 40 (eta: 329.9 / 1649.5 s) =========
------- 31.0    8.0     12.4    7.9     -------
========== 9 / 40 (eta: 389.3 / 1730.0 s) =========
------- 9.8     25.4    28.4    10.1    -------
========== 10 / 40 (eta: 462.9 / 1851.7 s) =========
-

# obj3 obs3

In [7]:
RTYPE = "indy7gripper"
# DAT_DIR = '20210907-071329'
FILE_OPTION = "obj_3c_obs3"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# # CNAME_LIST = ['Full'] #,'Pairwise']
# VISUALIZE = False
# PLAY_RESULT = False
# SHOW_STATE = False

# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# MAX_ITER = 10000

# GRASP_SAMPLE = 30
# STABLE_SAMPLE = 50
# SEARCH_SAMPLE_RATIO= 100


### test

In [8]:
I_START = 0
I_END = 40
N_DAT = I_END-I_START

In [9]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 6.2     5.7     8.6     6.3     -------
========== 1 / 40 (eta: 26.8 / 1072.6 s) =========
------- 16.1    14.5    8.3     9.1     -------
========== 2 / 40 (eta: 74.9 / 1498.4 s) =========
------- 106.3   103.7   82.2    93.5    -------
========== 3 / 40 (eta: 460.6 / 6141.6 s) =========
------- 6.4     11.1    13.9    8.4     -------
========== 4 / 40 (eta: 500.4 / 5004.4 s) =========
------- 4.9     5.7     5.3     5.5     -------
========== 5 / 40 (eta: 521.7 / 4173.9 s) =========
------- 5.4     10.2    6.5     5.5     -------
========== 6 / 40 (eta: 549.3 / 3662.2 s) =========
------- 30.7    6.9     5.1     11.6    -------
========== 7 / 40 (eta: 603.7 / 3449.6 s) =========
------- 9.5     27.7    12.0    11.8    -------
========== 8 / 40 (eta: 664.7 / 3323.7 s) =========
------- 10.1    8.0     7.4     5.7     -------
========== 9 / 40 (eta: 695.9 / 3093.1 s) =========
------- 21.6    7.3     5.6     7.2     -------
========== 10 / 40 (eta: 737.5 / 2950.0 s) =========
-

# obj3 obs5

In [10]:
RTYPE = "indy7gripper"
# DAT_DIR = '20210907-071329'
FILE_OPTION = "obj_3c_obs5"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# # CNAME_LIST = ['Full'] #,'Pairwise']
# VISUALIZE = False
# PLAY_RESULT = False
# SHOW_STATE = False

# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# MAX_ITER = 10000

# GRASP_SAMPLE = 30
# STABLE_SAMPLE = 50
# SEARCH_SAMPLE_RATIO= 100


### test

In [11]:
I_START = 0
I_END = 40
N_DAT = I_END-I_START

In [12]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 104.0   104.0   105.0   108.6   -------
========== 1 / 40 (eta: 421.7 / 16867.4 s) =========
------- 13.5    23.8    35.5    14.4    -------
========== 2 / 40 (eta: 508.9 / 10178.9 s) =========
------- 6.5     32.1    11.0    6.7     -------
========== 3 / 40 (eta: 565.2 / 7535.6 s) =========
------- 10.5    8.5     5.8     9.7     -------
========== 4 / 40 (eta: 599.8 / 5997.6 s) =========
------- 8.6     4.9     16.7    8.3     -------
========== 5 / 40 (eta: 638.3 / 5106.5 s) =========
------- 11.9    4.7     11.8    8.2     -------
========== 6 / 40 (eta: 674.9 / 4499.3 s) =========
------- 6.5     22.4    6.3     11.7    -------
========== 7 / 40 (eta: 721.8 / 4124.5 s) =========
------- 24.4    5.8     8.1     8.1     -------
========== 8 / 40 (eta: 768.2 / 3841.1 s) =========
------- 10.2    7.3     5.1     5.6     -------
========== 9 / 40 (eta: 796.5 / 3539.8 s) =========
------- 7.7     9.8     5.1     5.7     -------
========== 10 / 40 (eta: 824.7 / 3298.9 s) =======

# obj3 obs7

In [3]:
RTYPE = "indy7gripper"
# DAT_DIR = '20210907-071329'
FILE_OPTION = "obj_3c_obs7"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# # CNAME_LIST = ['Full'] #,'Pairwise']
# VISUALIZE = False
# PLAY_RESULT = False
# SHOW_STATE = False

# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# MAX_ITER = 10000

# GRASP_SAMPLE = 30
# STABLE_SAMPLE = 50
# SEARCH_SAMPLE_RATIO= 100


### test

In [4]:
I_START = 0
I_END = 40
N_DAT = I_END-I_START

In [ ]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 8.3     5.9     10.6    16.7    -------
========== 1 / 40 (eta: 41.5 / 1658.7 s) =========
------- 5.9     23.8    7.2     20.3    -------
========== 2 / 40 (eta: 98.7 / 1974.6 s) =========
------- 6.2     7.3     9.8     5.5     -------
========== 3 / 40 (eta: 127.6 / 1700.9 s) =========
------- 5.8     6.5     12.7    15.7    -------
========== 4 / 40 (eta: 168.3 / 1682.7 s) =========
------- 18.9    8.8     6.2     14.7    -------
========== 5 / 40 (eta: 216.9 / 1734.8 s) =========
------- 8.7     5.8     7.9     14.2    -------
========== 6 / 40 (eta: 253.5 / 1690.1 s) =========
------- 13.6    13.7    6.4     105.8   -------
========== 7 / 40 (eta: 392.9 / 2245.3 s) =========
------- 104.1   104.1   106.7   104.5   -------
========== 8 / 40 (eta: 812.3 / 4061.6 s) =========
------- 8.8     6.4     6.1     6.6     -------
========== 9 / 40 (eta: 840.3 / 3734.7 s) =========
------- 6.5     5.4     10.2    15.9    -------
========== 10 / 40 (eta: 878.3 / 3513.4 s) =========
-